In [1]:
!pip install transformers

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
%%capture
!apt install git-lfs

In [4]:
import os
import librosa
import gc
import ast
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import torchaudio
import sys
import datasets
from datasets import load_dataset,load_metric
from transformers import AutoFeatureExtractor
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
import librosa
from datasets import load_from_disk
from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import torch

2024-07-18 12:41:39.692440: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 12:41:39.692567: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 12:41:39.825093: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
model_checkpoint = "facebook/wav2vec2-base"
batch_size = 32

In [6]:
metric = load_metric("accuracy")

/tmp/ipykernel_35/2588679507.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [7]:
dataset = load_dataset("audiofolder", data_dir="/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm")

Resolving data files:   0%|          | 0/53868 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/10798 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/4634 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 53868
    })
    validation: Dataset({
        features: ['audio', 'label'],
        num_rows: 10798
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 4634
    })
})

In [9]:
from datasets import load_dataset

# Load the dataset
#dataset = load_dataset("audiofolder", data_dir="/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm")

# Function to get a subset of the dataset
def get_subset(dataset_split, percentage):
    split_size = len(dataset_split)
    subset_size = int(split_size * percentage)
    return dataset_split.shuffle(seed=42).select(range(subset_size))

# Define the percentage to select
percentage = 0.05

# Get subsets for train, validation, and test splits
train_subset = get_subset(dataset['train'], percentage)
validation_subset = get_subset(dataset['validation'], percentage)
test_subset = get_subset(dataset['test'], percentage)

# Create a new DatasetDict with the subsets but with the same names
subset_dataset = {
    'train': train_subset,
    'validation': validation_subset,
    'test': test_subset
}

# Optionally, convert to DatasetDict if needed
from datasets import DatasetDict
subset_dataset = DatasetDict(subset_dataset)

# Verify the sizes
print(f"Train subset size: {len(subset_dataset['train'])}")
print(f"Validation subset size: {len(subset_dataset['validation'])}")
print(f"Test subset size: {len(subset_dataset['test'])}")

# Now you can use subset_dataset for your tasks


Train subset size: 2693
Validation subset size: 539
Test subset size: 231


In [10]:
subset_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 2693
    })
    validation: Dataset({
        features: ['audio', 'label'],
        num_rows: 539
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 231
    })
})

In [12]:
dataset = subset_dataset

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 2693
    })
    validation: Dataset({
        features: ['audio', 'label'],
        num_rows: 539
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 231
    })
})

In [14]:
dataset["test"][8]

{'audio': {'path': '/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/testing/fake/file2003.wav_16k.wav_norm.wav_mono.wav_silence.wav',
  'array': array([0.03839111, 0.02700806, 0.03109741, ..., 0.05004883, 0.05895996,
         0.06265259]),
  'sampling_rate': 16000},
 'label': 0}

In [15]:
# Print the label names
print(dataset["train"].features["label"])

# Get the labels
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

print(id2label)

ClassLabel(names=['fake', 'real'], id=None)
{'0': 'fake', '1': 'real'}


In [83]:
# # Load the processor and model
# processor, model = load_wav2vec_model()

In [16]:
id2label

{'0': 'fake', '1': 'real'}

In [17]:
feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [19]:
max_duration = 5.0  # seconds

In [20]:
sampling_rate = 16000  # Example value, adjust as per your audio data


In [21]:
# Extract phoneme features using librosa
def extract_phoneme_features(audio, sr):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)

# Analyze prosody features
def analyze_prosody(audio, sr):
    pitches, magnitudes = librosa.core.piptrack(y=audio, sr=sr)
    pitch = pitches[magnitudes > np.median(magnitudes)]
    tempo, _ = librosa.beat.beat_track(y=audio, sr=sr)
    rms = librosa.feature.rms(y=audio)
    return np.array([np.mean(pitch), np.mean(tempo), np.mean(rms)])

# Detect unnatural pauses
def detect_pauses(audio, sr):
    silence = librosa.effects.split(y=audio, top_db=30)
    pauses = [(silence[i][0] - silence[i-1][1]) / sr for i in range(1, len(silence))]
    return np.array(pauses)

# Extract background noise features using librosa
def extract_background_noise(audio, sr):
    zcr = librosa.feature.zero_crossing_rate(y=audio)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)
    return np.array([np.mean(zcr), np.mean(spectral_centroid)])

# Identify click sounds and artifacts
def identify_clicks_and_artifacts(audio, sr):
    clicks = librosa.effects.clicks(times=None, frames=None, sr=sr)
    return np.array([np.sum(clicks)])

In [24]:
# Define the preprocessing function
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    sampling_rate = feature_extractor.sampling_rate

    # Extract Wav2Vec2 features
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=sampling_rate,
        max_length=int(sampling_rate * max_duration),
        truncation=True,
    )

    # Extract custom features
    phoneme_features = [extract_phoneme_features(audio, sampling_rate) for audio in audio_arrays]
    prosody_features = [analyze_prosody(audio, sampling_rate) for audio in audio_arrays]
    pauses = [detect_pauses(audio, sampling_rate) for audio in audio_arrays]
    background_noise = [extract_background_noise(audio, sampling_rate) for audio in audio_arrays]
    #clicks_and_artifacts = [identify_clicks_and_artifacts(audio, sampling_rate) for audio in audio_arrays]

    # Combine all features
    combined_features = [
        np.concatenate((inputs["input_values"][i], phoneme_features[i], prosody_features[i], pauses[i], background_noise[i]))#, clicks_and_artifacts[i]))
        for i in range(len(audio_arrays))
    ]

    # Convert to tensor
    inputs["input_values"] = combined_features

    return inputs

In [22]:
# def preprocess_function(examples):
#     audio_arrays = [x["array"] for x in examples["audio"]]
#     inputs = feature_extractor(
#         audio_arrays, 
#         sampling_rate=feature_extractor.sampling_rate, 
#         max_length=int(feature_extractor.sampling_rate * max_duration), 
#         truncation=True, 
#     )
#     return inputs

In [25]:
# Define which causing trouble next cell 
import librosa
import numpy as np

# Define your sampling rate
sampling_rate = 16000

# Example: Generate clicks every 1 second
duration = 10  # Total duration in seconds
times = np.array([1.0, 2.0, 3.0])  # Clicks at 1s, 2s, and 3s

# Generate clicks
clicks = librosa.clicks(times=times, sr=sampling_rate)

# 'clicks' is now a numpy array containing the audio signal with clicks

# You can use 'clicks' in your further processing






In [26]:
encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio"], batched=True)
encoded_dataset

Map:   0%|          | 0/2693 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Map:   0%|          | 0/539 [00:00<?, ? examples/s]

Map:   0%|          | 0/231 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values'],
        num_rows: 2693
    })
    validation: Dataset({
        features: ['label', 'input_values'],
        num_rows: 539
    })
    test: Dataset({
        features: ['label', 'input_values'],
        num_rows: 231
    })
})

In [27]:
num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
model_name = model_checkpoint.split("/")[-1]


args = TrainingArguments(
    f"{model_name}-finetuned-ks",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    
)

In [29]:
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [30]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [31]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
0,0.000000,nan,0.512059
1,0.000000,nan,0.512059
2,0.000000,nan,0.512059
4,0.000000,nan,0.512059


TrainOutput(global_step=105, training_loss=0.09029326665969122, metrics={'train_runtime': 1252.752, 'train_samples_per_second': 10.748, 'train_steps_per_second': 0.084, 'total_flos': 6.052184111659594e+17, 'train_loss': 0.09029326665969122, 'epoch': 4.94})

In [32]:
trainer.evaluate()

{'eval_loss': nan,
 'eval_accuracy': 0.5120593692022264,
 'eval_runtime': 22.0813,
 'eval_samples_per_second': 24.41,
 'eval_steps_per_second': 0.77,
 'epoch': 4.94}

In [37]:
trainer.push_to_hub()

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-66991dd2-6db96cc564c83d420d56645b;9fba4c9e-8344-45f4-9a7d-40e1a520f154)

Repository Not Found for url: https://huggingface.co/api/models/wav2vec2-base-finetuned-ks.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

In [34]:
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
  

In [38]:
import torch
import numpy as np
import librosa
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification


# Load the fine-tuned model and feature extractor
model_name = "motheecreator/wav2vec2-base-finetuned-ks"  # Replace with your model's name
model = AutoModelForAudioClassification.from_pretrained(model_name)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

# Set the model to evaluation mode
model.eval()

# Function to preprocess a single audio file
def preprocess_audio(file_path, feature_extractor, sampling_rate=16000, max_length=16000):
    # Load the audio file
    audio, sr = librosa.load(file_path, sr=sampling_rate)
    
    # Ensure the audio is the correct length
    if len(audio) > max_length:
        audio = audio[:max_length]  # Truncate if necessary
    else:
        audio = np.pad(audio, (0, max_length - len(audio)))  # Pad if necessary
    
    # Convert audio to the format expected by the model
    inputs = feature_extractor(
        audio,
        sampling_rate=sampling_rate,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt",
        truncation=True
    )
    return inputs

def predict_audio(file_path, model, feature_extractor, id2label):
    # Preprocess the audio file
    inputs = preprocess_audio(file_path, feature_extractor)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs).logits
        predicted_ids = torch.argmax(outputs, dim=-1).item()
    
    # Convert predicted ID to string for dictionary lookup
    predicted_id_str = str(predicted_ids)
    
    # Check if the predicted ID is in id2label
    if predicted_id_str in id2label:
        prediction = id2label[predicted_id_str]
    else:
        prediction = "Unknown label"
    
    return prediction

# Perform prediction on the single audio file
file_path = "/kaggle/input/the-fake-or-real-dataset/for-original/for-original/testing/real/file1004.wav"
prediction = predict_audio(file_path, model, feature_extractor, id2label)
print(f"The audio file is classified as: {prediction}")



config.json:   0%|          | 0.00/2.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

The audio file is classified as: real


In [39]:
# Perform prediction on the single audio file
file_path = "/kaggle/input/trebble/1714316769340.wav"
prediction = predict_audio(file_path, model, feature_extractor, id2label)
print(f"The audio file is classified as: {prediction}")

The audio file is classified as: real
